In [1]:
# Finger detection and counting 

In [2]:
import cv2
import numpy as np
from sklearn.metrics import pairwise

In [3]:
# Globals
# roi = region of interest
roi_top = 50
roi_bottom = 300
roi_left = 50
roi_right = 300

weight_for_background_accum = 0.5
frames_for_background_accum = 60

In [4]:
def accumulate_background(frame, background, accumulatation_weight):
   
    if background is None:
        background = frame.copy().astype("float")
        return background
    
    return cv2.accumulateWeighted(frame.astype("float"), background, accumulatation_weight)

In [5]:
def segment_hand(frame, background, threshold=25):
    
    difference = cv2.absdiff(background.astype("uint8"), frame)
    
    _, thresholded_img = cv2.threshold(difference, threshold, 255, cv2.THRESH_BINARY)
    
    thresholded_img = cv2.morphologyEx(thresholded_img, cv2.MORPH_OPEN, (5,5), iterations=2)
    
    image, contours, hierarchy = cv2.findContours(thresholded_img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return (thresholded_img, None)
    else:
        # Assumtion that the largest object is the hand
        hand_contours = max(contours, key=cv2.contourArea)
        
        return (thresholded_img, hand_contours)

In [6]:
def count_fingers(thresholded_img, hand_contours):
    
    convex_hull = cv2.convexHull(hand_contours)
    
    # Get the outermost points
    top    = tuple(convex_hull[convex_hull[:, :, 1].argmin()][0])
    bottom = tuple(convex_hull[convex_hull[:, :, 1].argmax()][0])
    left   = tuple(convex_hull[convex_hull[:, :, 0].argmin()][0])
    right  = tuple(convex_hull[convex_hull[:, :, 0].argmax()][0])

    # Approximation that the center of the hand is in the middle of these points
    center_x = left[0] + right[0] // 2
    center_y = top[1] + bottom[1] // 2
    
    # Find the longest distance from the hand center point to any outermost point (finger tip)
    distances = pairwise.euclidean_distances([(center_x, center_y)], Y=[left, right, top, bottom])[0]
    max_distance = distances.max()
    
    # Create a circle mask from the center point of the hand, with X% of the max distance
    radius = int(max_distance * 0.8)
    circumference = (2 * np.pi * radius)
    
    circular_mask = np.zeros(thresholded_img.shape[:2], dtype="uint8")
    cv2.circle(circular_mask, (center_x, center_y), radius, color=255, thickness=10)
    
    # Only look at the part of the thresholded image that is on the circle to find the fingers crossing it
    circular_roi = cv2.bitwise_and(thresholded_img, thresholded_img, mask=circular_mask)
    image, contours, hierarchy = cv2.findContours(circular_roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    finger_count = 0
    
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        
        # The contour is considered a finger if it is:
        #  not too low in the image (= points from wrist),
        #  not too small (= noice)
        #  not too large (= part of hand)
        not_too_low = (y + h) < center_y
        not_too_small = cv2.contourArea(contour) > 100
        not_too_large = contour.shape[0] < (circumference * 0.25)
        
        if not_too_low and not_too_large:
            finger_count += 1
    
    return (finger_count, circular_roi)

In [7]:
esc_key = 27
camera = cv2.VideoCapture(0)
processed_frames = 0
background = None

while True:
    is_read, frame = camera.read()
    
    frame = cv2.flip(frame, 1)
        
    frame_copy = frame.copy()
    cv2.rectangle(frame_copy, (roi_left,roi_top), (roi_right, roi_bottom), (0,0,255), 2)
    
    roi = frame.copy()[roi_top:roi_bottom, roi_left:roi_right]
    
    roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    cv2.GaussianBlur(roi_gray, (9,9), 0)
    
    if processed_frames < frames_for_background_accum:
        background = accumulate_background(roi_gray, background, weight_for_background_accum)
        cv2.putText(frame_copy, "Please wait, calculating background...", 
                    (roi_left, roi_top-10), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,0,255), 2)
    
    else:
        (thresholded_img, hand_contours) = segment_hand(roi_gray, background, 10)
        cv2.imshow("thresholded", thresholded_img)

        if not hand_contours is None:
            cv2.drawContours(roi_gray, hand_contours, -1, 255)
            cv2.imshow("Hand contours", roi_gray)
            
            finger_count, circular_roi = count_fingers(thresholded_img, hand_contours)
            cv2.imshow("Roi circular mask", circular_roi)
            
            cv2.putText(frame_copy, f"Fingers: {finger_count}", 
                    (roi_left, roi_top-10), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,0,255), 2)

    processed_frames += 1
    
    cv2.imshow("Count fingers", frame_copy)

    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == esc_key:
        break
    
camera.release()
cv2.destroyAllWindows()